# Contenido<a id='contenido'></a>
* [Introducción](#int)
* [Iniciación de datos](#ini)
    *  [Examina datos](#exa)
    *  [Descripción de datos](#des)
* [Machine Learning](#ml)
    *  [Segmentación de datos](#seg)
    *  [Entrenamiento](#ent)
        * [Algoritmo DecisionTreeClassifier](#alg1)
        * [Algoritmo RandomForestClassifier](#alg2)
        * [Algoritmo LogisticRegression](#alg3)
    *  [Calidad del modelo](#cal)
    *  [Prueba de cordura](#pru)
* [Conclusión general](#con)

# Introducción<a id='intr'></a>

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: **Smart o Ultra**. Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos. 

Para esta tarea de clasificación vamos a crear un modelo que escoja el plan correcto. 

Desarrollaremos un modelo con la mayor exactitud posible con al menos un umbral de exactitud del 75%. Usaremos el dataset para comprobar la exactitud.

## Objetivo

El objetivo es obtener un modelo capaz de predecir el tipo de plan Ultra o Smart que necesitan los usuarios.

# Iniciación de datos<a id='ini'></a>

Importamos las librerias:

In [1]:
# librerias generales
import pandas as pd
import numpy as np

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Cargamos el dataset:

In [2]:
# carga de datos
try:
    users = pd.read_csv('users_behavior.csv')
except:
    users = pd.read_csv('datasets/users_behavior.csv')


## Examina datos<a id='exa'></a>

In [3]:
users.head()


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
users.duplicated().sum()

0

El dataset contiene 3214 registros y 5 columnas, los tipos de datos son correctos, no tenemos valores ausentes ni registros duplicados.

## Descripción de los datos <a id='des'></a>


Cada observación en el dataset contiene información del comportamiento mensual sobre un usuario. La información dada es la siguiente:

- `сalls`: número de llamadas
- `minutes`: duración total de la llamada en minutos
- `messages`: número de mensajes de texto
- `mb_used`: Tráfico de Internet utilizado en MB
- `is_ultra`: plan para el mes actual (Ultra - 1, Smart - 0)

La columna is_ultra será nuestra variable dependiente / objetivo y nuestras variables independientes serán las otras columnas calls, minutes, messages y mb_used.

# Machine Learning<a id='ml'></a>

## Segmentación de datos

Antes que todo necesitamos segmentar nuestro dataset `users` en 3 partes con proporciones 3:1:1 de la siguiente manera:

- 60% de datos para entrenamiento
- 20% de datos para validación
- 20% de datos para prueba

In [6]:
# features variable independiente/caracteristicas
features = users.drop('is_ultra', axis=1)

# target variable dependiente/objetivo
target = users['is_ultra']

# Segmentación de datos entrenamiento 60%, validación 20% y prueba 20%
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.40, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.50, random_state=12345)

# Tamaño de los segmentos de datos
print('dataset', users.shape)
print('features_train:', features_train.shape)
print('target_train:', target_train.shape)

print('features_valid:', features_valid.shape)
print('target_valid:', target_valid.shape)

print('features_test:', features_test.shape)
print('target_test:', target_test.shape)


dataset (3214, 5)
features_train: (1928, 4)
target_train: (1928,)
features_valid: (643, 4)
target_valid: (643,)
features_test: (643, 4)
target_test: (643,)


## Entrenamiento<a id='ent'></a>

Realizaremos la creación de modelos usando 3 algoritmos de entrenamiento para clasificación e hiperparámetros que nos permitiran aprender de los datos y predecir nuevas observaciones.

Usaremos:
 
- Arbol de decision `DecisionTreeClassifier`, usaremos `max_depth` para determinar varias profundidades.
- Bosque aleatorio `RamdomForestClassifier`, usaremos `n_stimators` para determinar varios arboles y 'max_depth' varias profundidades.
- Regresion logística `LogisticRegression`, usaremos 'solver-liblinear', random_state=12345.

### Algoritmo DecisionTreeClassifier<a id='alg1'></a>

Vamos a entrenar un modelo con el algoritmo de Arbol de decision. Los hiperparámetros a usar son: 
- ramdom_state=12345
- max_depth= 1 a 5

In [7]:
# Arbol de decision con profundidad 10
best_est = 0
best_score = 0

for depth in range(1, 6):
    print('depth', depth)

    # creación de modelo de entrenamiento mediante arbol de decisión
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)

    # entrena el modelo con el conjunto de entrenamiento
    model.fit(features_train, target_train)

    # predicción para datos de entrenamiento
    prediction_train = model.predict(features_train)
    score_train = accuracy_score(target_train, prediction_train)
    print('Train data accuracy ', score_train)

    # prediccion para datos de valiación
    prediction_valid = model.predict(features_valid)
    score = accuracy_score(target_valid, prediction_valid)
    print('Valid data accuracy ', score)
    print()

    if score > best_score:
        best_score = score
        best_est = depth


print('RESULT: Best model with DecisionTreeClassifier on the validation dataset with depth = {} and accuracy {}'.format(best_est, best_score))

depth 1
Train data accuracy  0.7577800829875518
Valid data accuracy  0.7542768273716952

depth 2
Train data accuracy  0.7878630705394191
Valid data accuracy  0.7822706065318819

depth 3
Train data accuracy  0.8075726141078838
Valid data accuracy  0.7853810264385692

depth 4
Train data accuracy  0.8106846473029046
Valid data accuracy  0.7791601866251944

depth 5
Train data accuracy  0.8200207468879668
Valid data accuracy  0.7791601866251944

RESULT: Best model with DecisionTreeClassifier on the validation dataset with depth = 3 and accuracy 0.7853810264385692


Hemos realizado nuestro entrenamiento mediante clasificación por árbol de decision `DecisionTreeClassifier` variando la profundidad del árbol de 1 a 10. Obtuvimos que el mejor modelo tiene profundidad de 3 y su exactitud es del 78%. Además pudimos observar el comportamiento de las predicciones con los datos de entrenamiento y con los datos de validación, podemos notar que entre más profundidad le demos al árbol empezará a mostrarse un sobreajuste en los datos de entrenamiento, ya que los datos de entrenamientos tienen un mayor valor de precisión respecto a las predicciones realizadas en el conjunto de datos de validación. 

### Algoritmo RandomForestClassifier<a id='alg2'></a>

Usaremos el algoritmo de bosque aleatorio, con los hiperparámetros: 
- ramdom_state=12345
- n_stimators= 10 a 20
- max_depth= 1 a 3

In [8]:
# Bosque aleatorio
best_score = 0
best_tree = 0
best_depth = 0

# Variar el número de arboles tree y la profundidad de cada arbol depth
for tree in range(10, 21):
    for depth in range(1, 4):
        
        print('----------', tree,'tree -', 'depth', depth,'----------')

        # creación del modelo de entrenamiento mediante bosque aleatorio
        model = RandomForestClassifier(
            random_state=12345, n_estimators=tree, max_depth=depth)

        # entrena el modelo con el conjunto de entrenamiento
        model.fit(features_train, target_train)

        # prediccion para datos de entrenamiento
        score_train = model.score(features_train, target_train)
        print('Train data accuracy', score_train)

        # prediccion para datos de valiación
        score = model.score(features_valid, target_valid)
        print('Valid data accuracy', score)
        print()

        if score > best_score:
            best_score = score
            best_tree = tree
            best_depth = depth

print('RESULT: Best model with RandomForestClassifier on the validation dataset with trees:{}-depth:{} and accuracy {}'.format(best_tree, best_depth, best_score))

---------- 10 tree - depth 1 ----------
Train data accuracy 0.7442946058091287
Valid data accuracy 0.7558320373250389

---------- 10 tree - depth 2 ----------
Train data accuracy 0.7785269709543569
Valid data accuracy 0.7776049766718507

---------- 10 tree - depth 3 ----------
Train data accuracy 0.8101659751037344
Valid data accuracy 0.7853810264385692

---------- 11 tree - depth 1 ----------
Train data accuracy 0.7448132780082988
Valid data accuracy 0.7542768273716952

---------- 11 tree - depth 2 ----------
Train data accuracy 0.7883817427385892
Valid data accuracy 0.7853810264385692

---------- 11 tree - depth 3 ----------
Train data accuracy 0.8101659751037344
Valid data accuracy 0.7838258164852255

---------- 12 tree - depth 1 ----------
Train data accuracy 0.7442946058091287
Valid data accuracy 0.7527216174183515

---------- 12 tree - depth 2 ----------
Train data accuracy 0.7883817427385892
Valid data accuracy 0.7838258164852255

---------- 12 tree - depth 3 ----------
Train da

Hemos probado varios modelos de entrenamiento con el algoritmo bosque aleatorio `RandomForestClassifier` y obtuvimos el mejor modelo obtuvo un 79% de precisión usando 13 árboles con profundidad de 2. Además notamos que también se obtuvo un 79% de presición con el conjunto de datos de entremiento, por lo que hemos controlado que no exista un sobreajuste en nuestro modelo y que en el modelo anterior si ocurría.

### Algoritmo LogisticRegression<a id='alg3'></a>

El último algoritmo de aprendizaje que probaremos es el de regresión logística. Los hiperparámetros a usar son: 
- ramdom_state=12345
- solver= liblinear

In [9]:
# Regresion Logística
model = LogisticRegression(random_state=12345, solver='liblinear')

# entrena el modelo con el conjunto de entrenamiento
model.fit(features_train, target_train)

# calcula la precisión de la predicción con los datos de entrenamiento y luego con los datos de validación
score_train = model.score(features_train, target_train)
score_valid = model.score(features_valid, target_valid)

print("Accuracy of the Logistic Regression Model in the training dataset", score_train)
print("Accuracy of the Logistic Regression Model in the validation dataset:", score_valid)


Accuracy of the Logistic Regression Model in the training dataset 0.7510373443983402
Accuracy of the Logistic Regression Model in the validation dataset: 0.7573872472783826


Con la regresión logística tenemos que nuestro modelo obtuvo un 76% de exactitud, siendo menor que los modelos anteriores. También podemos observar que la predicción con el conjunto de entrenamiento no se adelanta a la predición con el conjunto de validación, es decir no existe un sobreajuste por parte de los datos de entrenamiento.

De acuerdo a los modelos revisados anteriormente obtuvimos que el mejor modelo tiene las siguientes características:


    - Algoritmo de aprendizaje: RandomForestClassifier
    - hiperparametros a usar 13 arboles y profundidad 2.

## Calidad del modelo <a id='cal'></a>
Comprobaremos la calidad de nuestro mejor modelo usando ahora usando nuestro conjunto de prueba.

In [10]:
# creación del modelo de entrenamiento
best_model = RandomForestClassifier(random_state=12345, n_estimators=13, max_depth=2)

# entrena el modelo
best_model.fit(features_train, target_train)

# Guardamos nuestras prediciones del modelo con los datos de prueba
predictions_test = best_model.predict(features_test)

# Métrica de evaluación del modelo
score_test = accuracy_score(target_test, predictions_test)
print('Accuracy of the best model RandomForestClassifier on the test dataset', score_test)

Accuracy of the best model RandomForestClassifier on the test dataset 0.7744945567651633


Hemos usado nuestro modelo (RandomForestClassifier con 13 arboles con profundidad de 2) y obtuvimos las predicciones del dataset de prueba obteniendo un 77% de exactitud, siendo este 2 puntos porcentuales menor que lo obtenido en nuestro dataset de validación.

## Prueba de cordura <a id='pru'> </a>

Para demostrar que nuestro modelo funciona mejor que la casualidad, realizaremos una prueba de cordura. Esta prueba buscará problemas de clasificación en nuestro modelo por lo que vamos a comparar nuestras predicciones con prediciones basadas en la aleatoriedad y demostrar que nuestro modelo es mejor.

Dicho esto, simularemos respuestas de manera aleatoria para evaluar su precisión, entonces crearemos una serie del mismo tamaño de observaciones que nuestro conjunto Objetivo de prueba y  rellenaremos con 0 y 1 de forma aleatoria.  

In [11]:
# random.seed nos permite establecer como parametro '12345' como generador de números pseudoaleatorios
np.random.seed(12345)

# Creamos una serie de valores 1 y 0 aleatorias del tamaño de nuestro target de prueba
random_test =  pd.Series(np.random.choice([0, 1], size=len(target_test)))

# Revisamos como quedaron distribuidos nuestros 0 y 1
print(random_test.value_counts(dropna=False))
print()
print(random_test)

1    328
0    315
dtype: int64

0      0
1      1
2      1
3      1
4      0
      ..
638    0
639    1
640    1
641    0
642    1
Length: 643, dtype: int32


Como vemos `random_test` contiene respuestas 0 y 1 de manera aleatoria. Para concretar nuestra prueba de cordura vamos a medir la exactitud de las predicciones aleatorias

In [12]:
print('Accuracy of the random responses:', accuracy_score(target_test, random_test))
print('Accuracy of the model RandomForestClassifier', score_test)

Accuracy of the random responses: 0.4821150855365474
Accuracy of the model RandomForestClassifier 0.7744945567651633


Hemos medido la exactitud de las respuestas aleatorias y obtuvimos un 48%, claramente nuestro modelo de predicción es mejor ya que tenemos un 77% de exactitud, es decir nuestro modelo ha pasado la prueba de cordura ya que predice mejor que predicciones al azar.

# Conclusión general<a id='con'></a>

Realizamos una investigación sobre los algoritmos de clasificación para crear modelos predictivos variando sus hiperparámetros. De lo cual obtuvimos:

- Con `DecisionTreeClassifier` variamos la profundidad del árbol de 1 a 10. Obtuvimos que el mejor modelo tiene profundidad de 3 y su exactitud es del 78%, sin embargo las predicciones en el conjunto de entrenamiento estuvo sobreajustado.

- Con `RandomForestClassifier` obtuvimos el mejor modelo con un 79% de precisión usando 13 árboles con profundidad de 2, las predicciones en el conjunto de entrenamiento no presentó sobreajuste por lo que es un buen indicativo.

- Con `LogisticRegression` nuestro modelo obtuvo un 76% de exactitud, siendo menor que los modelos anteriores. También podemos observar que la predicción con el conjunto de entrenamiento no se adelanta a la predición con el conjunto de validación, es decir no existe un sobreajuste por parte de los datos de entrenamiento.

Por lo tanto, podemos determinar que el modelo de predicción usa el algoritmo de aprendizaje de bosque aleatorio `RandomForestClassifier`. El bosque está compuesta de 13 árboles con profundidad de 2. La calidad del modelo fue medida mediante la exactitud del 77% superando el umbral de exactitud planteado inicialmente en el objetivo de este análisis y además superando la prueba de cordura.

Hemos obtenido un modelo capaz de predecir el tipo de plan (Ultra o Smart) que necesitan los usuarios de la compañía Megaline. 

